In [ ]:

#### keep this since it combines all data
import os
import glob
import pandas as pd

def detect_header_rows(filepath, sheet_name, keywords, lookahead=10):
    preview = pd.read_excel(filepath, sheet_name=sheet_name, nrows=lookahead, header=None)
    last_header_row = -1
    for i, row in preview.iterrows():
        for cell in row:
            if pd.notna(cell) and any(keyword.lower() in str(cell).lower() for keyword in keywords):
                last_header_row = max(last_header_row, i)
    if last_header_row >= 0:
        return list(range(last_header_row + 1))
    else:
        raise ValueError("Could not detect headers using provided keywords.")

def read_excel_with_detected_header(filepath, sheet_name, keywords=["(kt)", "kt CO2 equivalent", "GREENHOUSE GAS"], flatten=True):
    header_rows = detect_header_rows(filepath, sheet_name, keywords)
    df = pd.read_excel(filepath, sheet_name=sheet_name, header=header_rows)
    if flatten and isinstance(df.columns, pd.MultiIndex):
        def clean_column(col):
            parts = [str(part).strip() for part in col if pd.notna(part)]
            keep_parts = [p for p in parts if any(unit in p for unit in ['SINK CATEGORIES', 'CO2', 'CH4', 'N2O', 'SF6', 'HFC', 'PFC', '(kt)', 'NF', 'NO', 'NMVOC', 'CO', 'SO'])]
            return ' '.join(keep_parts).strip()
        df.columns = [clean_column(col) for col in df.columns]
    return df

def extract_year_from_filename(filename):
    parts = filename.split('_')
    if len(parts) > 2:
        try:
            return int(parts[2])
        except ValueError:
            return None
    return None

# Paths and config
folder_path = "data"
sheet_name = "Summary1.As1"
output_folder = os.path.join(folder_path, "Summary1As1")
os.makedirs(output_folder, exist_ok=True)
ghg_keywords = ['CO2', 'CH4', 'N2O', 'SF6', 'HFC', 'PFC']
category_col = "GREENHOUSE GAS SOURCE AND SINK CATEGORIES"
header_keywords = ["(kt)", "kt CO2 equivalent", "GREENHOUSE GAS"]

# Step 1: Process each Excel file, save processed CSV
for filepath in glob.glob(os.path.join(folder_path, "*.xlsx")):
    try:
        print(f"Processing: {filepath}")
        df = read_excel_with_detected_header(filepath, sheet_name=sheet_name, keywords=header_keywords)

        categories = df[category_col]
        numeric_df = df.drop(columns=[category_col]).apply(pd.to_numeric, errors='coerce')
        df_clean = pd.concat([categories, numeric_df], axis=1)
        df_clean = df_clean.dropna(axis=0, how='all', subset=numeric_df.columns)
        df_clean = df_clean.dropna(axis=1, how='all')

        cols_to_keep = [category_col] + [col for col in df_clean.columns if any(k in col for k in ghg_keywords)]
        df_ghg = df_clean[cols_to_keep]

        year = extract_year_from_filename(os.path.basename(filepath))
        if year is not None:
            df_ghg = df_ghg.copy()  # make an explicit copy to avoid the warning
            df_ghg.loc[:, 'Year'] = year

        else:
            print(f"Warning: Could not extract year from filename {filepath}")

        output_name = os.path.splitext(os.path.basename(filepath))[0] + "_processed.csv"
        df_ghg.to_csv(os.path.join(output_folder, output_name), index=False)

    except Exception as e:
        print(f"Error processing {filepath}: {e}")

# Step 2: Combine all processed CSVs into one DataFrame
all_processed_files = glob.glob(os.path.join(output_folder, "*_processed.csv"))
df_list = [pd.read_csv(f) for f in all_processed_files]
combined_df = pd.concat(df_list, ignore_index=True)

# Step 3: Summarize emissions by year
emission_cols = [col for col in combined_df.columns if col not in [category_col, 'Year']]
summary_by_year = combined_df.groupby('Year')[emission_cols].sum()

print("Summary of emissions by year:")
print(summary_by_year)

# Optional: save combined summary CSV
combined_summary_csv = os.path.join(output_folder, "Summary1As1_combined.csv")
combined_df.to_csv(combined_summary_csv, index=False)
print(f"Combined CSV saved to {combined_summary_csv}")


In [5]:
##perfect works all good
import os
import glob
import pandas as pd

def detect_header_rows(filepath, sheet_name, keywords, lookahead=10):
    preview = pd.read_excel(filepath, sheet_name=sheet_name, nrows=lookahead, header=None)
    last_header_row = -1
    for i, row in preview.iterrows():
        for cell in row:
            if pd.notna(cell) and any(keyword.lower() in str(cell).lower() for keyword in keywords):
                last_header_row = max(last_header_row, i)
    if last_header_row >= 0:
        return list(range(last_header_row + 1))
    else:
        raise ValueError("Could not detect headers using provided keywords.")

# Using your preferred read_excel_with_detected_header and clean_column
def read_excel_with_detected_header(filepath, sheet_name, keywords=["(kt)", "kt CO2 equivalent", "GREENHOUSE GAS"], flatten=True):
    header_rows = detect_header_rows(filepath, sheet_name, keywords)
    df = pd.read_excel(filepath, sheet_name=sheet_name, header=header_rows)
    if flatten and isinstance(df.columns, pd.MultiIndex):
        def clean_column(col):
            parts = [str(part).strip() for part in col if pd.notna(part)]
            # This is your original cleaning logic
            keep_parts = [p for p in parts if any(unit in p for unit in ['SINK CATEGORIES', 'CO2', 'CH4', 'N2O', 'SF6', 'HFC', 'PFC', '(kt)', 'NF', 'NO', 'NMVOC', 'CO', 'SO'])]
            return ' '.join(keep_parts).strip()
        df.columns = [clean_column(col) for col in df.columns]
    return df

def extract_year_from_filename(filename):
    parts = filename.split('_')
    if len(parts) > 2:
        try:
            return int(parts[2])
        except ValueError:
            return None
    return None

# Paths and config
folder_path = "data"
sheet_name = "Summary1.As3"
output_folder = os.path.join(folder_path, "Summary1As3")
os.makedirs(output_folder, exist_ok=True)
ghg_keywords = ['CO2', 'CH4', 'N2O', 'SF6', 'HFC', 'PFC']
# Note: category_col will be dynamically found based on keywords
header_keywords = ["(kt)", "kt CO2 equivalent", "GREENHOUSE GAS"]


# Update the dictionary keys to match the exact text from your data
expected_rows_per_subcategory = {
    'TOTAL NATIONAL EMISSIONS AND REMOVALS': 1,
    '1. ENERGY': 12,
    '2.  INDUSTRIAL PROCESSES AND PRODUCT USE': 9,
    '3.  AGRICULTURE': 11,  # Added extra space after dot
    '4.  LAND USE, LAND-USE CHANGE AND FORESTRY  (4)': 9,  # Added extra space after dot and (4)
    '5.  WASTE': 6,  # Added extra space after dot
    '6.  OTHER   (PLEASE SPECIFY)(6)': 1,  # Added extra spaces and (6)
    'INTERNATIONAL BUNKERS': 3,
    'MULTILATERAL OPERATIONS': 1,
    'CO2 EMISSIONS FROM BIOMASS': 1,
    'CO2 CAPTURED': 1,
    'LONG-TERM STORAGE OF C IN WASTE DISPOSAL SITES': 1,
    'INDIRECT N2O': 1,
    'INDIRECT CO2': 1
}

# Update the mapping dictionary to match
subcategory_name_map = {
    'TOTAL NATIONAL EMISSIONS AND REMOVALS': 'Total',
    '1. ENERGY': 'Energy',
    '2.  INDUSTRIAL PROCESSES AND PRODUCT USE': 'Industrial processes and product use',
    '3.  AGRICULTURE': 'Agriculture',
    '4.  LAND USE, LAND-USE CHANGE AND FORESTRY  (4)': 'Land use, land-use change and forestry',
    '5.  WASTE': 'Waste',
    '6.  OTHER   (PLEASE SPECIFY)(6)': 'Other',
    'INTERNATIONAL BUNKERS': 'International bunkers',
    'MULTILATERAL OPERATIONS': 'Multilateral operations',
    'CO2 EMISSIONS FROM BIOMASS': 'CO2 emissions from biomass',
    'CO2 CAPTURED': 'CO2 captured',
    'LONG-TERM STORAGE OF C IN WASTE DISPOSAL SITES': 'Long-term storage of C in waste disposal sites',
    'INDIRECT N2O': 'Indirect N2O',
    'INDIRECT CO2': 'Indirect CO2'
}



# Step 1: Process each Excel file, save processed CSV
for filepath in glob.glob(os.path.join(folder_path, "*.xlsx")):
    try:
        print(f"Processing: {filepath}")
        # Use your preferred read_excel_with_detected_header
        df = read_excel_with_detected_header(filepath, sheet_name=sheet_name, keywords=header_keywords)

        # Find the category column dynamically after cleaning
        category_column = None
        for col in df.columns:
            if 'GREENHOUSE GAS' in col.upper() and ('SOURCE' in col.upper() or 'SINK' in col.upper() or 'CATEGORIES' in col.upper()):
                category_column = col
                break

        if category_column is None:
            print(f"Warning: Could not find category column in {filepath}")
            continue

        # Initialize new category columns
        df['Main_Category'] = "Total national emissions and removals" # Main category is constant for this sheet
        df['Sub_Category'] = "Other" # Default
        df['Detail_Category'] = df[category_column] # Detail is the original cleaned text

        current_sub_category = "Other"
        rows_in_current_block = 0
        expected_rows = 0

        # Iterate through rows to assign Sub_Category based on position
        for index, row in df.iterrows():
            category_text = str(row[category_column]).strip().upper()

            # Check if the row text matches a known sub-category header
            matched_header = None
            # Use exact match for header detection
            if category_text in expected_rows_per_subcategory.keys():
                 matched_header = category_text

            if matched_header:
                # Found a sub-category header row
                current_sub_category = subcategory_name_map.get(matched_header, "Other")
                expected_rows = expected_rows_per_subcategory.get(matched_header, 1) # Default to 1 if not in map
                rows_in_current_block = 1 # This row is the first in the block
                df.loc[index, 'Sub_Category'] = current_sub_category
                # Detail_Category is already set to original text
                # print(f"--- Matched Header: '{matched_header}' at row {index}. Expected rows: {expected_rows} ---") # Debug print

            elif rows_in_current_block > 0 and rows_in_current_block < expected_rows:
                # This row is part of the current sub-category block
                df.loc[index, 'Sub_Category'] = current_sub_category
                rows_in_current_block += 1
                # Detail_Category is already set to original text

            else:
                # This row is not a recognized header and not part of a known block
                # It will keep the default 'Other' sub_category and its original text as detail
                # Only print warning if the text is not empty or 'nan' or a note
                original_category_text = str(row[category_column]).strip()
                if pd.notna(row[category_column]) and original_category_text != '' and 'Note:' not in original_category_text:
                     print(f"Warning: Row {index} with text '{original_category_text}' (Cleaned: '{category_text}') did not match a known block pattern. Assigned to 'Other' subcategory.")
                current_sub_category = "Other" # Reset state
                rows_in_current_block = 0
                expected_rows = 0
                # Sub_Category and Detail_Category are already defaulted


        # Ensure category_column is included before dropping
        cols_to_drop_from_numeric = [col for col in ['Main_Category', 'Sub_Category', 'Detail_Category'] if col in df.columns]
        if category_column in df.columns:
             cols_to_drop_from_numeric.append(category_column)

        categories = df[[category_column, 'Main_Category', 'Sub_Category', 'Detail_Category']]
        numeric_df = df.drop(columns=cols_to_drop_from_numeric).apply(pd.to_numeric, errors='coerce')

        # Align indices before concatenating
        numeric_df = numeric_df.reindex(df.index)
        df_clean = pd.concat([categories, numeric_df], axis=1)

        df_clean = df_clean.dropna(axis=0, how='all', subset=numeric_df.columns)
        df_clean = df_clean.dropna(axis=1, how='all')

        # Keep category columns and GHG-related columns
        category_cols = [category_column, 'Main_Category', 'Sub_Category', 'Detail_Category']
        cols_to_keep = category_cols + [col for col in df_clean.columns if col not in category_cols and any(k in col.upper() for k in ghg_keywords)]
        df_ghg = df_clean[cols_to_keep]

        year = extract_year_from_filename(os.path.basename(filepath))
        if year is not None:
            df_ghg = df_ghg.copy()
            df_ghg.loc[:, 'Year'] = year
        else:
            print(f"Warning: Could not extract year from filename {filepath}")

        output_name = os.path.splitext(os.path.basename(filepath))[0] + "_processed.csv"
        df_ghg.to_csv(os.path.join(output_folder, output_name), index=False)

        print(f"Saved: {output_name}")
        print(f"Categories preserved: {len(df_ghg)}")
        print("---")

    except Exception as e:
        print(f"Error processing {filepath}: {e}")

# Step 2: Combine all processed CSVs into one DataFrame
all_processed_files = glob.glob(os.path.join(output_folder, "*_processed.csv"))
if all_processed_files:
    df_list = [pd.read_csv(f) for f in all_processed_files]
    combined_df = pd.concat(df_list, ignore_index=True)

    # Step 3: Summarize emissions with proper hierarchy
    # Find the category column in combined data (using the original name for consistency)
    category_column_original = None
    for col in combined_df.columns:
        if 'GREENHOUSE GAS' in col.upper() and ('SOURCE' in col.upper() or 'SINK' in col.upper() or 'CATEGORIES' in col.upper()):
            category_column_original = col
            break

    if category_column_original:
        emission_cols = [col for col in combined_df.columns if col not in ['Main_Category', 'Sub_Category', 'Detail_Category', 'Year', category_column_original]]

        # Summary by main category and year (will always be the same main category for this sheet)
        summary_main = combined_df.groupby(['Main_Category', 'Year'])[emission_cols].sum()
        print("Summary by main category and year:")
        print(summary_main.head(10))

        # Summary by sub-category and year
        summary_sub = combined_df.groupby(['Main_Category', 'Sub_Category', 'Year'])[emission_cols].sum()
        print("\nSummary by sub-category and year:")
        print(summary_sub.head(10))

        # Summary by detail-category and year
        summary_detail = combined_df.groupby(['Main_Category', 'Sub_Category', 'Detail_Category', 'Year'])[emission_cols].sum()
        print("\nSummary by detail-category and year:")
        print(summary_detail.head(10))


        # Save outputs
        combined_summary_csv = os.path.join(output_folder, "Summary1As3_combined_hierarchical.csv")
        combined_df.to_csv(combined_summary_csv, index=False)
        print(f"\nCombined hierarchical CSV saved to {combined_summary_csv}")

        main_summary_csv = os.path.join(output_folder, "Summary1As3_main_categories.csv")
        summary_main.to_csv(main_summary_csv)
        print(f"Main category summary saved to {main_summary_csv}")

        sub_summary_csv = os.path.join(output_folder, "Summary1As3_sub_categories.csv")
        summary_sub.to_csv(sub_summary_csv)
        print(f"Sub-category summary saved to {sub_summary_csv}")

        detail_summary_csv = os.path.join(output_folder, "Summary1As3_detail_categories.csv")
        summary_detail.to_csv(detail_summary_csv)
        print(f"Detail-category summary saved to {detail_summary_csv}")

    else:
        print("Could not find category column in combined data for summarization")
else:
    print("No processed files found to combine")


Processing: data\GBR_2023_1990_13042023_164306.xlsx
Saved: GBR_2023_1990_13042023_164306_processed.csv
Categories preserved: 5
---
Processing: data\GBR_2023_1991_13042023_164356.xlsx
Saved: GBR_2023_1991_13042023_164356_processed.csv
Categories preserved: 5
---
Processing: data\GBR_2023_1992_13042023_164447.xlsx
Saved: GBR_2023_1992_13042023_164447_processed.csv
Categories preserved: 5
---
Processing: data\GBR_2023_1993_13042023_164537.xlsx
Saved: GBR_2023_1993_13042023_164537_processed.csv
Categories preserved: 5
---
Processing: data\GBR_2023_1994_13042023_164627.xlsx
Saved: GBR_2023_1994_13042023_164627_processed.csv
Categories preserved: 5
---
Processing: data\GBR_2023_1995_13042023_164718.xlsx
Saved: GBR_2023_1995_13042023_164718_processed.csv
Categories preserved: 5
---
Processing: data\GBR_2023_1996_13042023_164809.xlsx
Saved: GBR_2023_1996_13042023_164809_processed.csv
Categories preserved: 5
---
Processing: data\GBR_2023_1997_13042023_164900.xlsx
Saved: GBR_2023_1997_13042023_1

In [ ]:
import pandas as pd

extreme_weather = pd.read_excel('data/EM-DATA/public_emdat_custom_request_2025-06-11_b91b764b-7189-4747-8449-5320a1f25aed.xlsx')
extreme_weather_uk = extreme_weather[extreme_weather['ISO'] == 'GBR']

hazards = ['Wildfire', 'Flood', 'Drought', 'Heatwave', 'Extreme temperature', 'Storm']
extreme_weather_uk = extreme_weather_uk[extreme_weather_uk['Disaster Type'].isin(hazards)]

extreme_weather_uk



,DisNo.,Historic,Classification Key,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,External IDs,Event Name,ISO,...,Reconstruction Costs ('000 US$),"Reconstruction Costs, Adjusted ('000 US$)",Insured Damage ('000 US$),"Insured Damage, Adjusted ('000 US$)",Total Damage ('000 US$),"Total Damage, Adjusted ('000 US$)",CPI,Admin Units,Entry Date,Last Update
8,1990-0152-GBR,Yes,nat-met-sto-sto,Natural,Meteorological,Storm,Storm (General),NaN,NaN,GBR,...,NaN,NaN,NaN,NaN,NaN,NaN,41.652196,NaN,2003-07-01,2023-09-25
32,1990-0717-GBR,Yes,nat-met-sto-sto,Natural,Meteorological,Storm,Storm (General),NaN,Daria,GBR,...,NaN,NaN,3050000.0,7322543.0,3400000.0,8162835.0,41.652196,NaN,2007-05-08,2023-09-25
42,1990-0718-GBR,Yes,nat-met-sto-sto,Natural,Meteorological,Storm,Storm (General),NaN,Herta,GBR,...,NaN,NaN,NaN,NaN,10000.0,24008.0,41.652196,NaN,2007-05-09,2023-09-25
48,1990-0719-GBR,Yes,nat-met-sto-sto,Natural,Meteorological,Storm,Storm (General),NaN,Judith,GBR,...,NaN,NaN,30000.0,72025.0,60000.0,144050.0,41.652196,NaN,2008-10-21,2023-09-25
52,1990-0720-GBR,Yes,nat-met-sto-sto,Natural,Meteorological,Storm,Storm (General),NaN,Nana,GBR,...,NaN,NaN,30000.0,72025.0,70000.0,168058.0,41.652196,NaN,2007-05-09,2023-09-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1575,2022-0465-GBR,No,nat-met-ext-hea,Natural,Meteorological,Extreme temperature,Heat wave,NaN,NaN,GBR,...,NaN,NaN,NaN,NaN,NaN,NaN,93.294607,NaN,2022-07-28,2023-09-26
1606,2022-9405-GBR,No,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,NaN,NaN,GBR,...,NaN,NaN,NaN,NaN,NaN,NaN,93.294607,NaN,2022-08-19,2023-09-26
1636,2023-0438-GBR,No,nat-met-ext-hea,Natural,Meteorological,Extreme temperature,Heat wave,NaN,NaN,GBR,...,NaN,NaN,NaN,NaN,NaN,NaN,97.134993,NaN,2024-08-16,2024-10-01
1684,2023-0724-GBR,No,nat-met-sto-ext,Natural,Meteorological,Storm,Extra-tropical storm,NaN,Storm 'Ciaran',GBR,...,NaN,NaN,NaN,NaN,NaN,NaN,97.134993,NaN,2023-11-10,2023-12-07


,DisNo.,Historic,Classification Key,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,External IDs,Event Name,ISO,...,Reconstruction Costs ('000 US$),"Reconstruction Costs, Adjusted ('000 US$)",Insured Damage ('000 US$),"Insured Damage, Adjusted ('000 US$)",Total Damage ('000 US$),"Total Damage, Adjusted ('000 US$)",CPI,Admin Units,Entry Date,Last Update
8,1990-0152-GBR,Yes,nat-met-sto-sto,Natural,Meteorological,Storm,Storm (General),NaN,NaN,GBR,...,NaN,NaN,NaN,NaN,NaN,NaN,41.652196,NaN,2003-07-01,2023-09-25
32,1990-0717-GBR,Yes,nat-met-sto-sto,Natural,Meteorological,Storm,Storm (General),NaN,Daria,GBR,...,NaN,NaN,3050000.0,7322543.0,3400000.0,8162835.0,41.652196,NaN,2007-05-08,2023-09-25
42,1990-0718-GBR,Yes,nat-met-sto-sto,Natural,Meteorological,Storm,Storm (General),NaN,Herta,GBR,...,NaN,NaN,NaN,NaN,10000.0,24008.0,41.652196,NaN,2007-05-09,2023-09-25
48,1990-0719-GBR,Yes,nat-met-sto-sto,Natural,Meteorological,Storm,Storm (General),NaN,Judith,GBR,...,NaN,NaN,30000.0,72025.0,60000.0,144050.0,41.652196,NaN,2008-10-21,2023-09-25
52,1990-0720-GBR,Yes,nat-met-sto-sto,Natural,Meteorological,Storm,Storm (General),NaN,Nana,GBR,...,NaN,NaN,30000.0,72025.0,70000.0,168058.0,41.652196,NaN,2007-05-09,2023-09-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1575,2022-0465-GBR,No,nat-met-ext-hea,Natural,Meteorological,Extreme temperature,Heat wave,NaN,NaN,GBR,...,NaN,NaN,NaN,NaN,NaN,NaN,93.294607,NaN,2022-07-28,2023-09-26
1606,2022-9405-GBR,No,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,NaN,NaN,GBR,...,NaN,NaN,NaN,NaN,NaN,NaN,93.294607,NaN,2022-08-19,2023-09-26
1636,2023-0438-GBR,No,nat-met-ext-hea,Natural,Meteorological,Extreme temperature,Heat wave,NaN,NaN,GBR,...,NaN,NaN,NaN,NaN,NaN,NaN,97.134993,NaN,2024-08-16,2024-10-01
1684,2023-0724-GBR,No,nat-met-sto-ext,Natural,Meteorological,Storm,Extra-tropical storm,NaN,Storm 'Ciaran',GBR,...,NaN,NaN,NaN,NaN,NaN,NaN,97.134993,NaN,2023-11-10,2023-12-07


In [ ]:
## clean this when you have time
# 
import pandas as pd
import os

# Define specific file paths for each combined CSV
combined_files = {
    "Summary1As1": "data/Summary1As1/Summary1As1_combined.csv",
    "Summary1As2": "data/Summary1As2/Summary1As2_combined.csv",
    "Summary1As3": "data/Summary1As3/Summary1As3_combined.csv"
}

# Create a list to hold all dataframes
all_data = []

# Read each specific combined file
for sheet_name, file_path in combined_files.items():
    try:
        df = pd.read_csv(file_path)
        print(f"\nReading {sheet_name} from {os.path.basename(file_path)}")
        print(f"Original columns: {df.columns.tolist()}")
        
        # More flexible column standardization - if column contains "CO2", standardize it
        new_columns = []
        for col in df.columns:
            if 'CO2' in str(col) and 'equivalent' not in str(col).lower():
                new_columns.append('Net CO2 emissions/removals (kt)')
            else:
                new_columns.append(col)
        df.columns = new_columns
        
        print(f"Standardized columns: {df.columns.tolist()}")
        
        df["Sheet"] = sheet_name  # Add column to identify sheet
        all_data.append(df)
    except Exception as e:
        print(f"Error reading {file_path}: {e}")

# Combine all sheets into one DataFrame
combined_df = pd.concat(all_data, ignore_index=True)

# Check structure
print("\nCombined shape:", combined_df.shape)
print("Final columns:", combined_df.columns.tolist())

# Rename for clarity if needed
combined_df = combined_df.rename(columns={
    "GREENHOUSE GAS SOURCE AND SINK CATEGORIES": "Category"
})

# Check for missing Year values before melting
print("Year column info:")
print(combined_df['Year'].value_counts().sort_index())
print("Missing years:", combined_df['Year'].isna().sum())

# Fill missing years if needed (you might want to handle this differently)
# For now, let's see what's happening
combined_df_clean = combined_df.dropna(subset=['Year'])

# Reshape using melt: unpivot gases into long format
long_df = pd.melt(
    combined_df_clean,
    id_vars=["Category", "Year", "Sheet"],
    var_name="Gas",
    value_name="Emissions_kt"
)

# Convert Year to int if it's not already
long_df['Year'] = long_df['Year'].astype(int)

# Drop rows where Emissions_kt is NaN
long_df = long_df.dropna(subset=["Emissions_kt"])

# Check for missing years in final data
print("Years in final data:", sorted(long_df['Year'].unique()))
print("Year range:", long_df['Year'].min(), "to", long_df['Year'].max())

# Final preview
print(long_df.head())


long_df.to_csv("data/long_form_data.csv", index=False)


Reading Summary1As1 from Summary1As1_combined.csv
Original columns: ['GREENHOUSE GAS SOURCE AND SINK CATEGORIES', 'Net CO2 emissions/removals (kt)', 'CH4', 'N2O', 'HFCs(1) (kt CO2 equivalent)', 'PFCs(1)', 'SF6 (kt)', 'Year']
Standardized columns: ['GREENHOUSE GAS SOURCE AND SINK CATEGORIES', 'Net CO2 emissions/removals (kt)', 'CH4', 'N2O', 'HFCs(1) (kt CO2 equivalent)', 'PFCs(1)', 'SF6 (kt)', 'Year']

Reading Summary1As2 from Summary1As2_combined.csv
Original columns: ['GREENHOUSE GAS SOURCE AND SINK CATEGORIES', 'CO2 (kt)', 'CH4', 'N2O', 'Year']
Standardized columns: ['GREENHOUSE GAS SOURCE AND SINK CATEGORIES', 'Net CO2 emissions/removals (kt)', 'CH4', 'N2O', 'Year']

Reading Summary1As3 from Summary1As3_combined.csv
Original columns: ['GREENHOUSE GAS SOURCE AND SINK CATEGORIES', 'Net CO2 (kt)', 'CH4', 'N2O', 'Year']
Standardized columns: ['GREENHOUSE GAS SOURCE AND SINK CATEGORIES', 'Net CO2 emissions/removals (kt)', 'CH4', 'N2O', 'Year']

Combined shape: (1476, 9)
Final columns: [

KeyError: 'Sub_Category'

In [ ]:
## Clean and reshape the combined hierarchical data
import pandas as pd
import os

# Define the specific file path for the combined hierarchical CSV
# This should match the output name from the previous processing step
combined_hierarchical_file = {"Summary1As1" :"data/Summary1As1/Summary1As1_combined_hierarchical.csv",
                            "Summary1As2" :"data/Summary1As2/Summary1As2_combined_hierarchical.csv",
                            "Summary1As3" :"data/Summary1As3/Summary1As3_combined_hierarchical.csv"}
                              


# Create a list to hold the dataframe (only one file to read now)
all_data = []

# Read each specific combined file
for sheet_name, file_path in combined_hierarchical_file.items():
    # Read the combined hierarchical file
    try:
        # Read the CSV, ensuring pandas doesn't misinterpret headers if they look multi-level
        # Since our previous step flattened and added explicit columns, a simple read_csv is fine.
        df = pd.read_csv(file_path)
        print(f"\nReading {sheet_name} from {os.path.basename(file_path)}")
        print(f"Original columns: {df.columns.tolist()}")

        # --- Keep your column standardization logic ---
        # This is still useful if gas column names vary slightly across original sheets
        new_columns = []
        for col in df.columns:
            # Keep the new category columns and Year as they are
            if col in ['Main_Category', 'Sub_Category', 'Detail_Category', 'Year', 'GREENHOUSE GAS SOURCE AND SINK CATEGORIES']:
                new_columns.append(col)
            # Apply standardization to columns containing 'CO2' but not 'equivalent'
            elif 'CO2' in str(col) and 'equivalent' not in str(col).lower():
                new_columns.append('Net CO2 emissions/removals (kt)')
            else:
                new_columns.append(col)
        df.columns = new_columns

        print(f"Standardized columns: {df.columns.tolist()}")

        # Add a 'Sheet' column if you still need to distinguish original sheets
        # Note: Our previous code processed files one by one and saved them,
        # then combined. If you need the original sheet name, you might need
        # to add it during the first processing step before combining.
        # For now, we'll add a placeholder or skip if not needed.
        # If the previous step added 'Sheet', it will already be in the columns.
        if 'Sheet' not in df.columns:
            # Assuming the combined file doesn't have it, add a placeholder or derive if possible
            # For Summary1As1 combined, maybe just label it as such
            df["Sheet"] = "Summary1As1_Combined"


        all_data.append(df) # Add the single dataframe to the list

    except Exception as e:
        print(f"Error reading {combined_hierarchical_file}: {e}")
        # Exit or handle the error if the file couldn't be read
        exit() # Or sys.exit(1) if you import sys

# Combine all sheets into one DataFrame (already combined, so this just prepares for next steps)
# This step is technically redundant if only one file is read, but kept for structure
combined_df = pd.concat(all_data, ignore_index=True)


# Check structure
print("\nCombined shape:", combined_df.shape)
print("Final columns:", combined_df.columns.tolist())

# --- No need to rename 'GREENHOUSE GAS SOURCE AND SINK CATEGORIES' to 'Category' ---
# We already have 'Main_Category', 'Sub_Category', 'Detail_Category' and the original category column

# Check for missing Year values before melting
print("Year column info:")
print(combined_df['Year'].value_counts().sort_index())
print("Missing years:", combined_df['Year'].isna().sum())

# Fill missing years if needed (you might want to handle this differently)
# For now, let's see what's happening
# Drop rows where 'Year' is missing (or handle as needed)
# Drop rows where 'Year' is missing
combined_df_clean = combined_df.dropna(subset=['Year'])

# Promote 'Total' rows in Sub_Category to become their own Main_Category
if 'Sub_Category' in combined_df_clean.columns and 'Main_Category' in combined_df_clean.columns:
    total_mask = combined_df_clean['Sub_Category'] == 'Total'
    combined_df_clean.loc[total_mask, 'Main_Category'] = 'Total'
    combined_df_clean.loc[total_mask, 'Sub_Category'] = None

# Promote true subcategories from Detail_Category into Sub_Category
top_level_categories = [
    '1. Energy',
    '2.  Industrial processes and product use',
    '3.  Agriculture',
    '4.  Land use, land-use change and forestry  (4)',
    '5.  Waste',
    'International bunkers',
    'CO2 emissions from biomass'
]

if 'Detail_Category' in combined_df_clean.columns and 'Sub_Category' in combined_df_clean.columns:
    promote_mask = combined_df_clean['Detail_Category'].isin(top_level_categories)
    combined_df_clean.loc[promote_mask, 'Sub_Category'] = combined_df_clean.loc[promote_mask, 'Detail_Category']
    combined_df_clean.loc[promote_mask, 'Detail_Category'] = None

# Reshape using melt: unpivot gases into long format
# --- Update id_vars to include the new category columns ---
# Include the original category column too if you want to keep it
id_vars_list = ["Main_Category", "Sub_Category", "Detail_Category", "Year", "Sheet"]

# Ensure the original category column is included if it exists
original_cat_col_name = None
for col in combined_df_clean.columns:
     if 'GREENHOUSE GAS' in col.upper() and ('SOURCE' in col.upper() or 'SINK' in col.upper() or 'CATEGORIES' in col.upper()):
          original_cat_col_name = col
          break

if original_cat_col_name and original_cat_col_name not in id_vars_list:
    id_vars_list.append(original_cat_col_name)

# Filter id_vars_list to only include columns actually present in the DataFrame
id_vars_present = [col for col in id_vars_list if col in combined_df_clean.columns]

# Identify value_vars (emission columns) - exclude id_vars
value_vars_list = [col for col in combined_df_clean.columns if col not in id_vars_present]


long_df = pd.melt(
    combined_df_clean,
    id_vars=id_vars_present,
    value_vars=value_vars_list, # Explicitly list value columns
    var_name="Gas",
    value_name="Emissions_kt"
)

# Convert Year to int if it's not already
long_df['Year'] = long_df['Year'].astype(int)

# Drop rows where Emissions_kt is NaN (these are likely empty cells in the original data)
long_df = long_df.dropna(subset=["Emissions_kt"])

# Check for missing years in final data
print("Years in final data:", sorted(long_df['Year'].unique()))
print("Year range:", long_df['Year'].min(), "to", long_df['Year'].max())

# Final preview
print("\nPreview of long-form data:")
print(long_df.head())

# Save the final long-form data
long_form_output_path = "data/long_form_hierarchical.csv" # Give it a new name
long_df.to_csv(long_form_output_path, index=False)
print(f"\nLong-form hierarchical data saved to {long_form_output_path}")




Reading Summary1As1 from Summary1As1_combined_hierarchical.csv
Original columns: ['GREENHOUSE GAS SOURCE AND SINK CATEGORIES', 'Main_Category', 'Sub_Category', 'Detail_Category', 'Net CO2 emissions/removals (kt)', 'CH4', 'N2O', 'HFCs(1) (kt CO2 equivalent)', 'PFCs(1)', 'SF6 (kt)', 'Year']

Reading Summary1As2 from Summary1As2_combined_hierarchical.csv
Original columns: ['GREENHOUSE GAS SOURCE AND SINK CATEGORIES', 'Main_Category', 'Sub_Category', 'Detail_Category', 'CO2 (kt)', 'CH4', 'N2O', 'Year']

Reading Summary1As3 from Summary1As3_combined_hierarchical.csv
Original columns: ['GREENHOUSE GAS SOURCE AND SINK CATEGORIES', 'Main_Category', 'Sub_Category', 'Detail_Category', 'Net CO2 (kt)', 'CH4', 'N2O', 'Year']

Long-form hierarchical data saved to data/long_form_hierarchical.csv


In [ ]:
import pandas as pd

import pandas as pd

extreme_weather = pd.read_excel('data/EM-DATA/public_emdat_custom_request_2025-06-11_b91b764b-7189-4747-8449-5320a1f25aed.xlsx')
extreme_weather_uk = extreme_weather[extreme_weather['ISO'] == 'GBR']

hazards = ['Wildfire', 'Flood', 'Drought', 'Heatwave', 'Extreme temperature', 'Storm']
extreme_weather_uk = extreme_weather_uk[extreme_weather_uk['Disaster Type'].isin(hazards)]

extreme_weather_uk


extreme_weather = pd.read_excel('data/EM-DATA/public_emdat_custom_request_2025-06-11_b91b764b-7189-4747-8449-5320a1f25aed.xlsx')
extreme_weather_uk = extreme_weather[extreme_weather['ISO'] == 'GBR']

hazards = ['Wildfire', 'Flood', 'Drought', 'Heatwave', 'Extreme temperature', 'Storm']
extreme_weather_uk = extreme_weather_uk[extreme_weather_uk['Disaster Type'].isin(hazards)]

extreme_weather_uk


extreme_weather_uk['year'] = extreme_weather_uk['Start Year'].astype(int)
summary = (
    extreme_weather_uk
    .groupby(["year", "Disaster Type"])[
        ["Total Deaths", "Total Affected", "Total Damage ('000 US$)"]
    ]
    .sum()
    .reset_index()
)

summary
summary.to_csv('summary_extreme_weather_uk.csv', index=False)

In [ ]:

extreme_weather_uk['year'] = extreme_weather_uk['Start Year'].astype(int)
summary = (
    extreme_weather_uk
    .groupby(["year", "Disaster Type"])[
        ["Total Deaths", "Total Affected", "Total Damage ('000 US$)"]
    ]
    .sum()
    .reset_index()
)

summary
summary.to_csv('summary_extreme_weather_uk.csv', index=False)

In [1]:
import pandas as pd

def process_extreme_weather_data():
    # Read the data
    extreme_weather = pd.read_excel('data/EM-DATA/public_emdat_custom_request_2025-06-26_9e81f8a0-ef5b-49db-a30f-1546c51d9c78.xlsx')

    # Define hazards (keeping all countries)
    hazards = ['Wildfire', 'Flood', 'Drought', 'Heatwave', 'Extreme temperature', 'Storm']
    
    # Filter for hazards only (no country filter)
    filtered_weather = extreme_weather[
        extreme_weather['Disaster Type'].isin(hazards)
    ]
    
    # Convert year to integer
    filtered_weather['year'] = filtered_weather['Start Year'].astype(int)
    
    # Create summary using 'Country' instead of 'ISO'
    summary = (
        filtered_weather
        .groupby(["Country", "year", "Disaster Type"])[
            ["Total Deaths", "Total Affected", "Total Damage ('000 US$)"]
        ]
        .sum()
        .reset_index()
    )
    
    # Save to CSV
    summary.to_csv('data/EM-DATA/summary_extreme_weather_all_countries.csv', index=False)
    
    return summary

# Process the data
summary_data = process_extreme_weather_data()

# Print some basic information
print("Sample of the data:")
print(summary_data.head())
print("\nTotal number of countries:", summary_data['Country'].nunique())
print("\nList of countries:", sorted(summary_data['Country'].unique()))

C:\Users\Hodan\AppData\Local\Temp\ipykernel_20716\3916897635.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_weather['year'] = filtered_weather['Start Year'].astype(int)


Sample of the data:
   Country  year Disaster Type  Total Deaths  Total Affected  \
0  Albania  1992         Flood          11.0         35000.0   
1  Albania  1995         Flood           4.0          3500.0   
2  Albania  1997         Flood           0.0          6000.0   
3  Albania  2002         Flood           1.0         66884.0   
4  Albania  2002         Storm           6.0        125000.0   

   Total Damage ('000 US$)  
0                   7000.0  
1                      0.0  
2                      0.0  
3                  17500.0  
4                      0.0  

Total number of countries: 95

List of countries: ['Albania', 'Anguilla', 'Antigua and Barbuda', 'Argentina', 'Austria', 'Azores Islands', 'Bahamas', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Bermuda', 'Bolivia (Plurinational State of)', 'Bosnia and Herzegovina', 'Brazil', 'British Virgin Islands', 'Bulgaria', 'Canada', 'Cayman Islands', 'Chile', 'Colombia', 'Costa Rica', 'Croatia', 'Cuba', 'Czechia', 'Denmark', '